<a href="https://colab.research.google.com/github/dadebulba/DeepLearningProject/blob/main/PersonReID_TensorExtractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Learning Project - Person Re-Identification evaluation

[https://colab.research.google.com/github/dadebulba/DeepLearningProject/blob/main/PersonReID_TensorExtractor.ipynb](https://colab.research.google.com/github/dadebulba/DeepLearningProject/blob/main/PersonReID_TensorExtractor.ipynb)

Importing from Google Drive the dataset.zip and extract into dataset folder, change the path with your dataset location

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!unzip "/content/drive/MyDrive/UNITN/5° anno/Deep Learning 2021/dataset.zip" -d dataset

Import necessary libraries

In [2]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as T
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torchvision import transforms, utils
from torch.utils.data import Dataset, DataLoader
import os
from os import listdir
from os.path import isfile, join
from torch.utils.tensorboard import SummaryWriter
from PIL import Image
import random
import gc
random.seed(10)
# print cuda info
print(f"Cuda available: {torch.cuda.is_available()}")
print(f"Cuda device count: {torch.cuda.device_count()}")

Cuda available: True
Cuda device count: 1


## Define Siamese Network
This step is used to load the saved model and use it during testing 

In [3]:
class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()
        
    def forward(self, x):
        return x

class Siamese(nn.Module):

    def __init__(self, resnet):
        super(Siamese, self).__init__()
        self.resnet = resnet
        self.resnet.fc = Identity()
        self.linear = torch.nn.Sequential(
          torch.nn.Linear(in_features=2048, out_features=1024),
          torch.nn.Linear(in_features=1024, out_features=512),
          torch.nn.Sigmoid()
        )

    def forward_one(self, x):
        x = self.resnet(x)
        x = x.view(x.size()[0], -1)
        x = self.linear(x)
        return x

    def forward(self, x1, x2):
        out1 = self.forward_one(x1)
        out2 = self.forward_one(x2)
        return out1, out2

'''
Input arguments
  num_classes: number of classes in the dataset.
               This is equal to the number of output neurons.
'''
def initialize_resnet(num_classes):
  #load pre-trained resnet
  resnet = torchvision.models.resnet50(pretrained=True)
  num_features = resnet.fc.in_features
  resnet.fc = torch.nn.Sequential(
    torch.nn.Linear(in_features=num_features, out_features=1024),
    torch.nn.Linear(in_features=1024, out_features=512),
    torch.nn.Linear(in_features=512, out_features=num_classes),
    torch.nn.Sigmoid()
  )

  return resnet

In [4]:
class TestingDataset(Dataset):
  def __init__(self, images, dir):
      self.images = images
      self.dir = dir

  def __len__(self):
      return len(self.images)

  def __getitem__(self, idx):
      if torch.is_tensor(idx):
          idx = idx.tolist()

      img_name = self.images[idx]

      image = Image.open("%s/%s" % (self.dir, img_name))
      image = T.ToTensor()(image)
      image = F.interpolate(image, size=128)

      sample = (image, img_name)
      return sample

In [54]:
def compute_tensors(net, data_loader, target_dir):
  if not os.path.exists(target_dir):
    os.mkdir(target_dir)
  for idx, (image, image_name) in enumerate(data_loader):
    # Compute the forward pass
    tensor = image.to('cuda:0')
    tensor_to_save = net.forward_one(tensor)
    torch.save(tensor_to_save, "%s/%s.ph"%(target_dir, image_name[0].split(".")[0]))

In [55]:
def extract_tensors(device='cuda:0', img_root='./dataset'):
  # Instantiates the model
  net = initialize_resnet(num_classes=56).to(device)
  net = Siamese(net)
  net.load_state_dict(torch.load("/content/drive/MyDrive/UNITN/5° anno/Deep Learning 2021/models/siamese_net_reid_resnet50_5epoch.pth"))
  net.to(device)
  net.eval()
  query = [f for f in listdir("./dataset/queries")]
  test = [f for f in listdir("./dataset/test")]

  testing_dataset = TestingDataset(images=test, dir="./dataset/test")
  test_dataloader = torch.utils.data.DataLoader(testing_dataset, 1, shuffle=False, num_workers=4) #before num_workers=4

  query_dataset = TestingDataset(images=query, dir="./dataset/queries")
  query_dataloader = torch.utils.data.DataLoader(query_dataset, 1, shuffle=False, num_workers=2) #before num_workers=4

  compute_tensors(net=net, data_loader=query_dataloader, target_dir="./dataset/query_tensors")
  compute_tensors(net=net, data_loader=test_dataloader, target_dir="./dataset/test_tensors")

In [26]:
def main(threshold=0.036):

  query_tensors = [f for f in listdir("./dataset/query_tensors")]
  query_images = [f for f in listdir("./dataset/queries")]
  test_tensors = [f for f in listdir("./dataset/test_tensors")]
  test_images = [f for f in listdir("./dataset/test")]

  query_tensors.sort()
  query_images.sort()
  test_tensors.sort()
  test_images.sort()

  test_tensors_cuda = []
  for test in test_tensors:
    test_tensor = torch.load("{}/{}".format("./dataset/test_tensors", test))
    test_tensor.to('cuda:0')
    test_tensors_cuda.append(test_tensor)

  f = open("reid_results.txt", "w")

  for idxQ, query in enumerate(query_tensors):
    if idxQ == 10:
      break

    print(query, "doing")
    query_tensor = torch.load("{}/{}".format("./dataset/query_tensors", query))
    query_tensor.to('cuda:0')

    to_print = "{}:".format(query_images[idxQ])

    for idxT, test in enumerate(test_tensors_cuda):
      euclidean_distance = F.pairwise_distance(query_tensor, test)
      if euclidean_distance.item() < threshold:
        to_print = "{}{},".format(to_print, test_images[idxT])

    f.write(to_print[:-1])
    f.write("\n")

  f.close()
      

In [56]:
#This is done to prevent memory overflow and use only the computed tensors during test phase, if you have already the zip files with tensors you can skip this phase
extract_tensors()

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [32]:
## Make a zip containing the tensors if need to save them
#import shutil
#shutil.make_archive('query_tensors', 'zip', './dataset/query_tensors')
#shutil.make_archive('test_tensors', 'zip', './dataset/test_tensors')

'/content/test_tensors.zip'

In [27]:
main()

000000.ph doing
000001.ph doing
000002.ph doing
000003.ph doing
000004.ph doing
000005.ph doing
000006.ph doing
000007.ph doing
000008.ph doing
000009.ph doing
